In [1]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time

In [2]:
## get problem parameters and geometry
problem = problems.problem3

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F):
    
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*(G/KBTV)* Trace(F.trans*F ) + H


def Gel_energy_EDP(F): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    edp = (G/KBTV) * InnerProduct(F,dv) + H_prime * J * Trace(invF*dv)
    return edp


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = 0)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)
#F = Id(dim) + Grad(u)

## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((1,1,1))
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

phi = CF(-z)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(3)+ grad(u)
    if form == "Functional":
        #LHS
        
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx)
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    elif form == "EDP":
        BF += alpha * Gel_energy_EDP(F).Compile() * dx
        BF += delta * v[2]*dx
        BF += u[2]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[2]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    






In [ ]:
max_PG_it = 7
max_iter_qnewton = 15
max_iter_newton = 20
newton_damp = 0.5
softening_n = 15

tol_newton = 1e-5
tol_QN = 1e-5

gammas = np.flip(np.linspace(G_target, G_target*50 ,softening_n))
gammas = [G_target*10]
for num, load in enumerate(gammas):
    G.Set(load)
    for k in range(1,max_PG_it):
        print("PG it:",k)
        alpha.Set(2**k)
        psik.vec.data = psih.vec
        print("Starting QN")
        for i in range(max_iter_qnewton):
            with TaskManager():
                G_funs = u,v,alpha,delta,psih,psik 
                BF = Assemble_BF(BF,G_funs,form)
                
                if num ==0:
                    gfu = GridFunction(fes)
                    gfu.vec[:] = 0
                res = gfu.vec.CreateVector()
                w = gfu.vec.CreateVector()
                #iterations of newton for the NL problem at each QN step
                print("Solving nonlinear subproblem")
                for iter in range(max_iter_newton):
                    BF.Apply(gfu.vec,res)
                    BF.AssembleLinearization(gfu.vec)
                    inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                    w.data = newton_damp * inv * res
                    gfu.vec.data += -w
                    if abs(InnerProduct(w,res)) < tol_newton:
                        print("Tol achieved", InnerProduct(w,res))
                        break
                psih.vec.data = psih.vec + gfu.components[1].vec
                


            #update gfu_u, psih



PG it: 1
Starting QN
Solving nonlinear subproblem


In [4]:
Draw(gfu.components[0])

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene